---
# **Predicting stock market index using LSTM (N-day window)**

**Disclaimer**: Originally based on work by Hum Nath Bhandari, Binod Rimal, Nawa Raj Pokhrel, Ramchandra Rimal, Keshav Dahal, and Rajendra K.C. Khatri

**Reference**: Bhandari, H. N., Rimal, B., Pokhrel, N. R., Rimal, R., Dahal, K., & K C Khatri, R. (2022). Predicting stock market index using LSTM. Machine Learning with Applications, 9, 100320. https://doi.org/10.1016/j.mlwa.2022.100320

---

## **Introduction**

In this module we use various financial resources to create input data for our machine learning models. This includes the following data.
- S\&P 500 index data: Close price
- Vix index
- Interest Rate
- Unemployment Rate
- Consumer sentiment index
- Dollar Index
- MACD
- RSI
- ATR

#### **0. Define the number of days in the series**

In [ ]:
# Number of days in the sequence (e.g. 5, 21, 90)
Nd=21

#### **1. Importing basic libraries and APIs**

In [ ]:

# Mathematical library
import math

# Data manipulation libraries
import numpy as np
import pandas as pd

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.style.use('ggplot')

# Utility libraries
import warnings
warnings.filterwarnings('ignore')
import os
import datetime as dt


ModuleNotFoundError: No module named 'pandas'

#### **1. Data Loading and Verification**

##### **1.1 Loading input data**



In [ ]:
url = 'https://raw.githubusercontent.com/luiscunhacsc/course_dl_datasets/main/sp500_data.csv'
data = pd.read_csv(url)

data['Date']=pd.to_datetime(data['Date']).dt.date
data.set_index("Date",inplace=True)
data = data.iloc[:, 1:]



##### **1.2 Data Exploration**

In [ ]:
# Textual verification of the data
data.head()
data.tail()
print(data.shape)
data.describe()

In [ ]:
# Graphical verification of the data
df = data.copy()
df['MA_50'] = df['Close'].rolling(50).mean()
df['MA_200'] = df['Close'].rolling(200).mean()

fig = plt.figure(figsize=(10,5))
fig.set(facecolor = "white")
plt.plot(df['Close'], 'mediumblue',label=['S&P 500 Close Price'], linewidth = 2.2)
plt.plot(df['MA_50'],'red', label=['50-day MA'], linewidth = 2.2)
plt.plot(df['MA_200'],'darkgreen', label=['200-day MA'], linewidth = 2.2)
plt.legend(['S&P 500 Close Price', '50-day Moving Average', '200-day Moving Average'], loc='upper left')
plt.title('')
plt.xlabel('Time (years)')
plt.ylabel('Close price')
#fig.savefig(output_dir_path+ "original_data_plus_moving_averages.png",dpi=600)
plt.show()

#####  **1.3 Correlation heatmap**

In [ ]:
fig = plt.figure(figsize= (10,5))
sns.heatmap(data.corr(), annot=True)
sns.set_style("whitegrid")
#fig.savefig(output_dir_path+"correlation_heatmap2.png",dpi=600)
plt.show()

In [ ]:
fig = plt.figure(figsize = (20, 12))
data.plot(subplots = True, figsize = (12,10),grid=False)
sns.set_style("whitegrid")
#fig.savefig(output_dir_path+"timeseries.png",dpi=600)
plt.show()

#### **2. Prepare the data for the LSTM model - don't forget to set number of days (Nd) in step 0, above**

In [ ]:

# Store date information for plotting
dates = data.index

# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Define a function to create sequences of input data and labels
def create_sequences(data, sequence_length, target_column):
    x, y = [], []
    for i in range(len(data) - sequence_length - 1):
        x.append(data[i: i + sequence_length])
        y.append(data[i + sequence_length, target_column])
    return np.array(x), np.array(y)

# Create input sequences and labels
sequence_length = Nd  # Nd is the number of days in the sequence, to predict next day's (Nd+1) closing price   <--- "Nd" defined at the beginning of the script (step 0)
target_column = 0  # Assuming 'Close' is the first column in the dataset
x, y = create_sequences(data_scaled, sequence_length, target_column)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.20, shuffle=False)


#### **3. Define the LSTM model**

In [ ]:
input_layer = Input(shape=(sequence_length, data.shape[1]))
lstm_layer = LSTM(150)(input_layer)
output_layer = Dense(1)(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)

#### **4. Compile and train the model**

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mean_squared_error")

batch_size = 8
epochs = 50  # Adjust the number of epochs according to your needs
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/50
863/863 [==============================] - 22s 21ms/step - loss: 4.9936e-04
Epoch 2/50
863/863 [==============================] - 17s 20ms/step - loss: 3.6266e-05
Epoch 3/50
863/863 [==============================] - 18s 21ms/step - loss: 3.7898e-05
Epoch 4/50
863/863 [==============================] - 17s 20ms/step - loss: 3.0718e-05
Epoch 5/50
863/863 [==============================] - 17s 20ms/step - loss: 2.7502e-05
Epoch 6/50
863/863 [==============================] - 21s 24ms/step - loss: 2.8578e-05
Epoch 7/50
863/863 [==============================] - 17s 20ms/step - loss: 1.8668e-05
Epoch 8/50
863/863 [==============================] - 18s 21ms/step - loss: 2.2005e-05
Epoch 9/50
863/863 [==============================] - 17s 20ms/step - loss: 2.1363e-05
Epoch 10/50
863/863 [==============================] - 17s 19ms/step - loss: 1.9589e-05
Epoch 11/50
863/863 [==============================] - 18s 21ms/step - loss: 1.7108e-05
Epoch 12/50
863/863 [====================

#### **5. Make predictions using the trained model**

In [ ]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

#### **6. Trasform data back to original scale**

In [ ]:
# Inverse-transform the scaled data back to the original scale
y_train_true = scaler.inverse_transform(np.hstack(
    (y_train.reshape(-1, 1), np.zeros((y_train.shape[0], data.shape[1] - 1)))))
y_test_true = scaler.inverse_transform(np.hstack(
    (y_test.reshape(-1, 1), np.zeros((y_test.shape[0], data.shape[1] - 1)))))
y_train_pred = scaler.inverse_transform(np.hstack(
    (y_train_pred, np.zeros((y_train_pred.shape[0], data.shape[1] - 1)))))
y_test_pred = scaler.inverse_transform(
    np.hstack((y_test_pred, np.zeros((y_test_pred.shape[0], data.shape[1] - 1)))))




#### **7. Plot *all* the historical data, including predictions at the end**

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(dates, data['Close'], label='True Values', color='blue')
plt.plot(dates[sequence_length:len(y_train) + sequence_length],
         y_train_pred[:, 0], label='Train Predictions', color='green')
plt.plot(dates[len(y_train) + sequence_length:len(y_train) + len(y_test) +
         sequence_length], y_test_pred[:, 0], label='Test Predictions', color='red')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()


#### **8. Plot *only* the end of the series, containing the predictions**

In [ ]:
# Plot only the true values and test predictions
plt.figure(figsize=(12, 6))
plt.plot(dates[len(y_train) + sequence_length:], data['Close'][len(y_train) + sequence_length:], label='True Values (Test)', color='blue')
plt.plot(dates[len(y_train) + sequence_length:len(y_train) + len(y_test) + sequence_length], y_test_pred[:, 0], label='Test Predictions', color='red')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()
